# Wyjaśnialne uczenia maszynowe HW1

### Mateusz Sieniawski

W tym notebooku zaprezentowane jest rozwiązanie pomysłu "Impostor Forest". Tutaj przez impostorów będę rozumiał outliery. Impostor Forest jest lasem losowym. Na samym początku wybieramy jedną klasę, której nie będziemy używali w treningu. Na przykład, powiedzmy, że w naszych danych jest 9 klas (labeli). Wybieramy jedną klasę i zakładamy, że to są dane, których model wcześniej nie widział i ma je zaklasyfikować jako outliery podczas fazy testowania modelu. Następnie uczymy każde drzewo na trochę innym zbiorze danych. Z pozostałych 8 ośmiu klas wybieramy 2, którym zamieniamy klasę na "outlier" tzn. drzewo dostaje poprawnie zalabelowane 6 klas oraz dwie klasy przemianowane na outliery. Każde drzewo dostaje inny losowy zbiór przemianowanych klas.

Są dwie metryki do opisu jakości działania Impostor Foresta. Po pierwsze standardowe accuracy, a po drugie impostor detection rate - czyli jaki procent outlierów jest w stanie wykryć.

W tym notebooku uruchomiony zostaje Impostor Forest na datasecie https://archive.ics.uci.edu/ml/datasets/Avila , a także można go łatwo odpalić dla dwóch innych datasetów.

Impostor Forest osiąga gorsze accuracy niż zwykły Random Forest, ale potrafi wykrywać outliery (tzn. model może implice zwrócić predykcję "uważam że ten obiekt jest outlierem"). Dobierając odpowiednią wartość hiperparametrów można dobrać złoty środek pomiędzy accuracy a impostor detection rate.

Ten pomysł wymyśliłem sam i z moją najlepszą wiedzą nie został on nigdzie wczeniej opublikowany.

Najpewniej czuję się w Pythonie i najchętniej pracowałbym także w tym języku nad dużym projektem.

In [10]:
# The implementation is based on https://machinelearningmastery.com/implement-random-forest-scratch-python/
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import copy
import itertools
from sklearn.model_selection import train_test_split
import random

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

flatten = lambda t: [item for sublist in t for item in sublist]

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


def get_dataset_classes(dataset):
  classes = set()
  for row in dataset:
    label = row[-1]
    classes |= set([label])
  return classes


def split_dataset_by_class(dataset, classes_names):
  classes = dict()
  for class_name in classes_names:
    classes[class_name] = list()
  for row in dataset:
    label = row[-1]
    classes[label].append(row)
  return classes


def evaluate_impostor_entry_rate(dataset, algorithm, no_impostor_classes, *args):
  dataset = copy.deepcopy(dataset)
  classes = get_dataset_classes(dataset)
  no_dataset_classes = len(classes)
  impostor_label = int(max(classes)) + 1
  dataset_by_classes = split_dataset_by_class(dataset, classes)

  scores = []
  impostor_entry_accuracies = []
  for test_impostor_class in classes:
    dataset = copy.deepcopy(dataset)
    trees = list()
    test_impostor_data = dataset_by_classes[test_impostor_class]
    for row in test_impostor_data:
      dataset.remove(row)
    train, validation = train_test_split(dataset, test_size=0.2, random_state=42, shuffle=True)
    dataset_by_classes = split_dataset_by_class(train, classes)

    # Split for training data and impostor test data
    impostor_classes_combinations = itertools.combinations(list(classes - set([test_impostor_class])), no_impostor_classes)
    for impostor_classes in set(impostor_classes_combinations):
      not_impostors = flatten([dataset_by_classes[c] for c in classes if c not in impostor_classes and c != test_impostor_class])
      impostors = flatten([dataset_by_classes[c] for c in classes if c in impostor_classes])
      impostors_copy = copy.deepcopy(impostors)

      for i in range(len(impostors_copy)):
        impostors_copy[i][-1] = impostor_label
      data = not_impostors + impostors_copy

      test_set = list()
      for row in test_impostor_data:
          row_copy = list(row)
          test_set.append(row_copy)
          row_copy[-1] = None

      # Build tree
      tree = build_tree(data, max_depth, min_size, n_features)
      trees.append(tree)

    # Calculate imposter detection rate
    predicted = [bagging_predict(trees, row) for row in test_set]
    actual = [impostor_label for _ in range(len(test_impostor_data))]
    impostor_entry_accuracies.append(accuracy_metric(actual, predicted))

    # Calculate the standard accuracy
    validation_set = list()
    for row in validation:
        row_copy = list(row)
        validation_set.append(row_copy)
        row_copy[-1] = None
    predicted = [bagging_predict(trees, row) for row in validation_set]
    actual = [row[-1] for row in validation]
    scores.append(accuracy_metric(actual, predicted))

  return impostor_entry_accuracies, scores


# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

def preprocess_ecoli(dataset):
  dataset = dataset[1:]
  label_map = {'cp': 0,
               'im': 1,
               'pp': 2,
               'imU': 3,
               'om': 4,
               'omL': 5,
               'imL': 6,
               'imS': 7}
  for i in range(len(dataset)):
    dataset[i][-1] = label_map[dataset[i][-1]]
  return dataset

def preprocess_avila(dataset):
  label_map = {'A': 0,
               'B': 1,
               'C': 2,
               'D': 3,
               'E': 4,
               'F': 5,
               'G': 6,
               'H': 7,
               'I': 8,
               'W': 9,
               'X': 10,
               'Y': 11}
  for i in range(len(dataset)):
    dataset[i][-1] = label_map[dataset[i][-1]]
  return dataset

# Test the random forest algorithm
seed(2)
# load and prepare data
# https://archive.ics.uci.edu/ml/machine-learning-databases/glass/
#filename = '/content/glass.data'
# https://www.openml.org/d/39
#filename = '/content/dataset_ecoli.csv'
# https://archive.ics.uci.edu/ml/datasets/Avila
filename = 'content/avila-tr.txt'
dataset = load_csv(filename)
#dataset = preprocess_ecoli(dataset)
dataset = preprocess_avila(dataset)
#random.shuffle(dataset)
dataset = dataset[:500]
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
  str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))

for n_impostor_classes in range(5):
  for n_trees in [1, 5, 10]:

    impostor_entry_accuracies, scores = evaluate_impostor_entry_rate(dataset, random_forest, n_impostor_classes, max_depth, min_size, sample_size, n_trees, n_features)
    print('Impostor classes: %d' % n_impostor_classes)
    print('Trees: %d' % n_trees)
    print('Mean accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('Mean impostor detection accuracy: %.2f' % (sum(impostor_entry_accuracies)/float(len(impostor_entry_accuracies))))
    print("")


Impostor classes: 0
Trees: 1
Mean accuracy: 53.948%
Mean impostor detection accuracy: 0.00

Impostor classes: 0
Trees: 5
Mean accuracy: 51.337%
Mean impostor detection accuracy: 0.00

Impostor classes: 0
Trees: 10
Mean accuracy: 45.584%
Mean impostor detection accuracy: 0.00

Impostor classes: 1
Trees: 1
Mean accuracy: 58.855%
Mean impostor detection accuracy: 0.34

Impostor classes: 1
Trees: 5
Mean accuracy: 60.499%
Mean impostor detection accuracy: 0.13

Impostor classes: 1
Trees: 10
Mean accuracy: 62.298%
Mean impostor detection accuracy: 0.79

Impostor classes: 2
Trees: 1
Mean accuracy: 63.282%
Mean impostor detection accuracy: 5.68

Impostor classes: 2
Trees: 5
Mean accuracy: 64.985%
Mean impostor detection accuracy: 7.28

Impostor classes: 2
Trees: 10
Mean accuracy: 62.876%
Mean impostor detection accuracy: 5.25

Impostor classes: 3
Trees: 1
Mean accuracy: 55.771%
Mean impostor detection accuracy: 41.54

Impostor classes: 3
Trees: 5
Mean accuracy: 54.117%
Mean impostor detection 